In [ ]:
# Initial imports.
import pandas as pd
import numpy as np
from path import Path
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report

# Upload and Transform Data

In [ ]:
# Read the CSV from AWS.
from pyspark import SparkFiles
url ="https://YOUR-BUCKET-NAME.s3.amazonaws.com/District_data.csv"
spark.sparkContext.addFile(url)
districts = spark.read.csv(SparkFiles.get("District_data.csv"), sep=",", header=True, inferSchema=True)

In [ ]:
# Read the CSV file into a Pandas DataFrame
districts = pd.read_csv('District_data.csv')
districts = districts.replace('redacted',0)
districts.head()

In [ ]:
# Create a binary variable showing whether a district has above 50% proficiency or below.
districts['Math_metAbove50'] = np.where(districts['Math_metAbove'] >= 50, True, False)
districts['ELA_metAbove50'] = np.where(districts['ELA_metAbove'] >= 50, True, False)
districts['Math_metAbove50'] = districts['Math_metAbove50'].astype(int)
districts['ELA_metAbove50'] = districts['ELA_metAbove50'].astype(int)
districts.head(30)

In [ ]:
# Rename average years of teaching column.
districts = districts.rename(columns={"Avg Years Teaching (District)": "Avg_years_teaching"})
districts

In [ ]:
# Replace NaaNs with means.
districts = districts.fillna(districts.mean())

# Predicting ELA Proficiency

In [ ]:
# Define the feature set.
X = districts[["Enrollment","FRL_Perc","Teach_to_stud","Per_pupil_exp","Teacher_salary","Avg_years_teaching"]]
X.head()

In [ ]:
# Define the target set.
y = districts['ELA_metAbove50']
y[:5]

In [ ]:
# Splitting into Train and Test sets.
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=78)

In [ ]:
# Creating a StandardScaler instance.
scaler = StandardScaler()
# Fitting the Standard Scaler with the training data.
X_scaler = scaler.fit(X_train)

# Scaling the data.
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [ ]:
# Create a random forest classifier.
rf_model = RandomForestClassifier(n_estimators=500, random_state=78) 

In [ ]:
# Fitting the model
rf_model = rf_model.fit(X_train_scaled, y_train)

In [ ]:
# Making predictions using the testing data.
predictions = rf_model.predict(X_test_scaled)

In [ ]:
# Calculating the confusion matrix.
cm = confusion_matrix(y_test, predictions)

# Create a DataFrame from the confusion matrix.
cm_df = pd.DataFrame(
    cm, index=["Actual 0", "Actual 1"], columns=["Predicted 0", "Predicted 1"])

cm_df

In [ ]:
# Calculating the accuracy score.
acc_score = accuracy_score(y_test, predictions)
acc_score

In [ ]:
# Displaying results
print("Confusion Matrix")
display(cm_df)
print(f"Accuracy Score : {acc_score}")
print("Classification Report")
print(classification_report(y_test, predictions))

In [ ]:
# Calculate feature importance in the Random Forest model.
importances = rf_model.feature_importances_
importances

In [ ]:
# Sort the features by their importance.
sorted(zip(rf_model.feature_importances_, X.columns), reverse=True)

# Predicting Math Proficiency

In [ ]:
# Define the target set.
y = districts['Math_metAbove50']
y[:5]

In [ ]:
# Splitting into Train and Test sets.
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=78, stratify=True)

# Consider random state of 42.

In [ ]:
# Creating a StandardScaler instance.
scaler = StandardScaler()
# Fitting the Standard Scaler with the training data.
X_scaler = scaler.fit(X_train)

# Scaling the data.
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [ ]:
# Create a random forest classifier.
rf_model = RandomForestClassifier(n_estimators=500, random_state=78) 

In [ ]:
# Fitting the model
rf_model = rf_model.fit(X_train_scaled, y_train)

In [ ]:
# Making predictions using the testing data.
predictions = rf_model.predict(X_test_scaled)

In [ ]:
# Calculating the confusion matrix.
cm = confusion_matrix(y_test, predictions)

# Create a DataFrame from the confusion matrix.
cm_df = pd.DataFrame(
    cm, index=["Actual 0", "Actual 1"], columns=["Predicted 0", "Predicted 1"])

cm_df

In [ ]:
# Calculating the accuracy score.
acc_score = accuracy_score(y_test, predictions)

In [ ]:
# Displaying results
print("Confusion Matrix")
display(cm_df)
print(f"Accuracy Score : {acc_score}")
print("Classification Report")
print(classification_report(y_test, predictions))

In [ ]:
# Calculate feature importance in the Random Forest model.
importances = rf_model.feature_importances_
importances

In [82]:
# Sort the features by their importance.
sorted(zip(rf_model.feature_importances_, X.columns), reverse=True)

[(0.5016407578542948, 'FRL_Perc'),
 (0.13523997175285005, 'Per_pupil_exp'),
 (0.12407650312110306, 'Teacher_salary'),
 (0.09617499994012865, 'Enrollment'),
 (0.09260367004715042, 'Teach_to_stud'),
 (0.050264097284472856, 'Avg_years_teaching')]

In [ ]:
# Try imbalanced algorithm?